In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import string
import os
import time

driver = webdriver.Firefox()
driver.get('https://csf.tools/controlset/nist800-53r5/?name_filter=&baseline%5B%5D=nist800-53r5-high')



In [4]:
table_data = []
last_set = None
has_more_data = True

try:
    while [has_more_data]:
        table = WebDriverWait(driver=driver, timeout=10).until(
            EC.presence_of_element_located(locator=(By.ID, 'controlset-nist800-53r5'))
        )
        for row in table.find_elements(By. CSS_SELECTOR, 'tr'):
            columns = row.find_elements(By.CSS_SELECTOR, 'td')
            if len(columns) < 1:
                # likly the header, we should do more here later
                pass
            else:
                # Main code flow
                if "controlset-element-enhancement" in row.get_attribute("class"):
                    id = f'{last_set}{columns[0].text}'
                elif "controlset-row" in row.get_attribute("class"):
                    last_set = columns[0].text
                    id = last_set
                stride_column = columns[6].find_elements(By.CSS_SELECTOR, "span")
                stride_mapping = {}
                for letter in stride_column:
                    stride_mapping[letter.text] = "controlset-threat-yes" in letter.get_attribute("class")
                row_data = {
                    "id": id,
                    "name":  columns [1]. text,
                    "low":  True if  "•" in columns [2]. text else False,
                    "moderate": True if "•" in columns[3]. text else False,
                    "high": True if "•" in columns[4].text else False,
                    "spoofing": stride_mapping['S'],
                    "tampering":  stride_mapping['T'],
                    "repudiation": stride_mapping['R'],
                    "information_disclosure": stride_mapping['I'],
                    "denial_of_service":  stride_mapping['D'],
                    "elevation_of_privilege": stride_mapping['E'],
                    "lateral_movement":  stride_mapping['LM']
                }
                table_data.append(row_data)
        next_page = driver.find_elements(By.CSS_SELECTOR, "a.next.page-numbers")
        if len(next_page) > 0:
            print("Going to the next page")
            next_page.pop().click()
        else:
            print("out of data")
            has_more_data = False
            break

finally:
    driver.close()

Going to the next page
Going to the next page
Going to the next page
Going to the next page
Going to the next page
Going to the next page
Going to the next page
out of data


In [ ]:
import csv
with open(file="80053-STRIDELM-Mapping.csv", mode="w", newline="", encoding="utf-8") as output:
    writer = csv.DictWriter(f=output, fieldnames=table_data[0].keys())
    writer.writeheader()
    writer.writerows(table_data)

In [1]:
from NISTDataModel import Control, NIST_80053, CSF_Tools, Control_STRIDE

nist_data = NIST_80053.load_data(path_to_csv="NIST_SP-800-53_rev5_catalog_load.csv")
csf_tools_data = CSF_Tools(path_to_csv="80053-STRIDELM-Mapping.csv")


In [2]:
for ndata in nist_data:
    id = ndata.identifier
    csf = csf_tools_data.get(id=id)
    if csf:
        ndata.denial_of_service = csf.denial_of_service
        ndata.low = csf.low
        ndata.moderate = csf.moderate
        ndata.high = csf.high
        ndata.spoofing = csf.spoofing
        ndata.tampering = csf.tampering
        ndata.repudiation = csf.repudiation
        ndata.information_disclosure = csf.information_disclosure
        ndata.denial_of_service = csf.denial_of_service
        ndata.elevation_of_privilege = csf.elevation_of_privilege
        ndata.lateral_movement = csf.lateral_movement


Got nothing for id AC-2(6)
Got nothing for id AC-2(7)
Got nothing for id AC-2(8)
Got nothing for id AC-2(9)
Got nothing for id AC-2(10)
Got nothing for id AC-3(1)
Got nothing for id AC-3(2)
Got nothing for id AC-3(3)
Got nothing for id AC-3(4)
Got nothing for id AC-3(5)
Got nothing for id AC-3(6)
Got nothing for id AC-3(7)
Got nothing for id AC-3(8)
Got nothing for id AC-3(9)
Got nothing for id AC-3(10)
Got nothing for id AC-3(11)
Got nothing for id AC-3(12)
Got nothing for id AC-3(13)
Got nothing for id AC-3(14)
Got nothing for id AC-3(15)
Got nothing for id AC-4(1)
Got nothing for id AC-4(2)
Got nothing for id AC-4(3)
Got nothing for id AC-4(5)
Got nothing for id AC-4(6)
Got nothing for id AC-4(7)
Got nothing for id AC-4(8)
Got nothing for id AC-4(9)
Got nothing for id AC-4(10)
Got nothing for id AC-4(11)
Got nothing for id AC-4(12)
Got nothing for id AC-4(13)
Got nothing for id AC-4(14)
Got nothing for id AC-4(15)
Got nothing for id AC-4(16)
Got nothing for id AC-4(17)
Got nothing f

In [8]:
import csv

with open(file="Collection.csv", mode='w') as collected_output:
    writer = csv.DictWriter(f=collected_output, fieldnames=nist_data[1].dict().keys())
    writer.writeheader()
    for ndata in nist_data:
        writer.writerow(ndata.dict())
        

In [1]:
import pandas as pd

df = pd.read_csv('80053-STRIDELM-Mapping.csv')
df.to_parquet('output.parquet', compression='snappy')

